# Graph Management: Naming

In [1]:
# DL framework
import tensorflow as tf

from datetime import datetime

# common packages
import numpy as np
import os # handling file i/o
import sys
import math
import time # timing epochs

# for ordered dict when building layer components
import collections

# plotting pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import pyplot
from matplotlib import colors # making colors consistent
from mpl_toolkits.axes_grid1 import make_axes_locatable # colorbar helper

# read image
from imageio import imread
# + data augmentation
from scipy import ndimage
from scipy import misc

# used for manually saving best params
import pickle

# for shuffling data batches
from sklearn.utils import shuffle

# const
SEED = 42

# Helper to make the output consistent
def reset_graph(seed=SEED):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# helper to create dirs if they don't already exist
def maybe_create_dir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        print("{} createed".format(dir_path))
    else:
        print("{} already exists".format(dir_path))
    
# set log level to supress messages, unless an error
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Important Version information
print("Python: {}".format(sys.version_info[:]))
print('TensorFlow: {}'.format(tf.__version__))

# Check if using GPU
if not tf.test.gpu_device_name():
    print('No GPU found')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
    
reset_graph()

Python: (3, 5, 4, 'final', 0)
TensorFlow: 1.5.0-dev20171220
No GPU found


## Default graph

In [2]:
reset_graph()
# make constant
c_1 = tf.constant(1)

with tf.Session() as sess:
    # initialize global vars
    sess.run(tf.global_variables_initializer())
    
    # execute 
    output = sess.run(c_1)
    print(output)

1


Great, this works as expected.

### Inspecting the operations
We can use `get_operations()` on our graph to get all the operations

In [3]:
# print all operation information.  Since we're operating on the
# default graph, we can use the `tf.get_default_graph()` to grab our graph
for op in tf.get_default_graph().get_operations():
    print(op)

name: "Const"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT32
      tensor_shape {
      }
      int_val: 1
    }
  }
}

name: "init"
op: "NoOp"



In [4]:
# wrap into list for easier viewing
all_ops = [op for op in tf.get_default_graph().get_operations()]

# view all
print(all_ops)

[<tf.Operation 'Const' type=Const>, <tf.Operation 'init' type=NoOp>]


In [5]:
# view individual
print(all_ops[0])

name: "Const"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT32
      tensor_shape {
      }
      int_val: 1
    }
  }
}



In [6]:
# reset default graph
reset_graph()

# define graph (const x const)
c_2 = tf.constant(2)
c_3 = tf.constant(3)
m_1 = tf.multiply(c_2,c_3)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) 
    output = sess.run(m_1)
    print(output)
    
all_ops = [op for op in tf.get_default_graph().get_operations()]
print(all_ops)

6
[<tf.Operation 'Const' type=Const>, <tf.Operation 'Const_1' type=Const>, <tf.Operation 'Mul' type=Mul>, <tf.Operation 'init' type=NoOp>]


Note the name of th operations.  We see the `const` and `init` as expected, and we can assume the `Mul` is the new multiplication operation (as expected), but what's the `cons_1` operation?

In [7]:
# alternate view
for op in all_ops:
    print(op.name)

Const
Const_1
Mul
init


This is a good opportunity to show another function `get_operation_by_name()`.

In [8]:
# access the operation in multiple ways
print(all_ops[1])

print("=======================")

print(tf.get_default_graph().get_operation_by_name("Const_1"))

name: "Const_1"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT32
      tensor_shape {
      }
      int_val: 3
    }
  }
}

name: "Const_1"
op: "Const"
attr {
  key: "dtype"
  value {
    type: DT_INT32
  }
}
attr {
  key: "value"
  value {
    tensor {
      dtype: DT_INT32
      tensor_shape {
      }
      int_val: 3
    }
  }
}



Ok, so the operation is a constant, and the initial value is 3 (the second `constant`) call we make;
```
c_2 = tf.constant(2)
c_3 = tf.constant(3)
```
So why the `_1`? This is tensorflows way of making sure the name of the operations are unique -- you can read more [here](https://www.tensorflow.org/programmers_guide/graphs)